In [90]:
import pandas as pd, numpy as np, json
from glob import glob

In [25]:
glob('*')

['daily.csv',
 'hourly.csv',
 'read_data.ipynb',
 'stations.csv',
 'stations.json',
 'station_data',
 'weekdays.csv']

In [13]:
stations = pd.read_csv('stations.csv')

stations.columns = ['id', 'kpas', 'lon', 'lat', 'name']
stations.kpas = stations.kpas.round(0).astype(int)

In [24]:
stations.to_json('stations.json', orient='records', force_ascii=False)

In [28]:
hourly = pd.read_csv('hourly.csv')
weekdays = pd.read_csv('weekdays.csv')
daily = pd.read_csv('daily.csv')

In [46]:
daily.kpas = daily.kpas.round().astype(int)
daily = pd.pivot_table(daily, index=['station', 'date', 'name'], columns='type', values='kpas').reset_index()

In [96]:
def get_st(st, name, lon, lat, kpas):
    st_daily = daily.loc[daily.station == st, ['date','arrival','departure']
                   ].fillna(0
                   ).to_dict(orient='records')
    st_weekdays = weekdays.loc[weekdays.station == st, ['weekday', 'arrival', 'departure']
                         ].round(2
                         ).fillna(0
                         ).to_dict(orient='records')
    st_hourly = hourly.loc[hourly.station == st, ['hour', 'arrival', 'departure']
                     ].round(2
                     ).fillna(0
                     ).to_dict(orient='records')
    d = {
        'name': name,
        'id': st,
        'lon': lon,
        'lat': lat,
        'kpas': kpas,
        'daily': st_daily,
        'weekdays': st_weekdays,
        'hourly': st_hourly,
    }
    return st, d

all_st = dict(get_st(*t) for t in stations[['id', 'name', 'lon', 'lat', 'kpas']].values)

In [97]:
all_st[2201135]

{'name': 'Тарасівка',
 'id': 2201135,
 'lon': 30.318772999999997,
 'lat': 50.344643,
 'kpas': 1,
 'daily': [{'date': '2014-01-10', 'arrival': 0.0, 'departure': 1.0},
  {'date': '2014-01-16', 'arrival': 0.0, 'departure': 1.0},
  {'date': '2014-02-14', 'arrival': 0.0, 'departure': 1.0}],
 'weekdays': [{'weekday': 4.0, 'arrival': 0.0, 'departure': 1.0},
  {'weekday': 5.0, 'arrival': 0.0, 'departure': 1.0}],
 'hourly': [{'hour': 6.0, 'arrival': 0.0, 'departure': 1.0}]}

In [98]:
with open('all_stations.json', 'w', encoding='utf-8') as f:
    json.dump(all_st, f, ensure_ascii=False)